# Imports

In [1]:
import requests
from bs4 import BeautifulSoup as soup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import re
from datetime import datetime
import pandas as pd
from connection_keys import connection, alchemy_connection

### Setup para abrir o navegador com o Selenium e puxar a primeira página

In [2]:
#--| Setup
options = Options()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless')
options.add_argument('--disable-extensions')
#options.add_argument("--headless")
browser = webdriver.Chrome(executable_path=r'chromedriver.exe', options=options)
#--| Parse or automation
browser.get('https://www.kabum.com.br/hardware/placa-de-video-vga?pagina=1')
html = soup(browser.page_source, 'html.parser')
# browser.implicitly_wait(5)

### Configuiração para percorer as páginas e salvar todos dados com a taga referente ao produto em uma lista

In [8]:
pagina = 0
html_produtos = []
flag = 0

while flag == 0:
    items = html.findAll('div', {"class":"sc-fzqNqU jmuOAh"})
    if items != []:
        try:
            pagina = pagina + 1
            browser.get(f'https://www.kabum.com.br/hardware/placa-de-video-vga?pagina={pagina}')
            html = soup(browser.page_source, 'html.parser')
        except Exception: break
        html_produtos.append(items)
    else:
        break

### Tranformação da lista de listas em uma lista normal para percorer com mais facildiade os atributos do produto

In [9]:
flat_list = []
for sublist in html_produtos:
    for item in sublist:
        flat_list.append(item)

### Percorrendo a lista, capurando e salvando cada atributo no dicionario

In [10]:
now = datetime.now()
produtos = []

for container in flat_list:
    
    # Estreutura de para salvar
    marca = container.div.img['alt']
    nome_produto = container.find('a', {"class":"sc-fzoLsD gnrNhT item-nome"}).text

    if container.find('div', {"class":"sc-fzpjYC hKlXgM"}) != None:
        preco_antigo_cartao = (container.find('div', {"class":"sc-fzpjYC hKlXgM"}).text).split(" ")[2]
    else: 
        preco_antigo_cartao = '0'
    preco_atual_cartao =  (container.find('div', {"class":"sc-fznxsB ksiZrQ"}).text).split(" ")[1]
    preco_boleto = (container.find('div', {"class":"sc-fznWqX qatGF"}).text).split(" ")[1]
    
    image = container.find('div', {"class":"sc-fznKkj fEpBwY"})
    if "indisponivel" in image.img['src']:
        disponibilidade = 'Indisponivel'
    elif "disponivel" in image.img['src']:
        disponibilidade = 'Disponivel'

    produto = {
        "marca" : marca,
        "nome_produto": nome_produto,
        "preco_antigo_cartao": preco_antigo_cartao,
        "preco_atual_cartao": preco_atual_cartao,
        "preco_boleto": preco_boleto,
        "disponibilidade": disponibilidade,
        "data" : now.strftime("%Y-%m-%d"),
        "hora":  now.strftime("%H:%M:%S"),
        "loja": "Kabum" 
    }
    produtos.append(produto)


### Transformando o dict em Dataframe e visualizando uma amostra dos dados

In [11]:
df = pd.DataFrame(produtos)
df.sample(10)

,marca,nome_produto,preco_antigo_cartao,preco_atual_cartao,preco_boleto,disponibilidade,data,hora,loja
146,Galax,Placa de Vídeo Galax NVIDIA GeForce GTX 1650 E...,"1.446,94","1.399,88","1.189,90",Disponivel,2020-09-15,20:28:24,Kabum
109,Sapphire,Placa de Vídeo Sapphire Nitro+ AMD Radeon RX 5...,0,"2.352,82","1.999,90",Disponivel,2020-09-15,20:28:24,Kabum
165,Gainward,Placa de Vídeo Gainward NVIDIA GeForce RTX 206...,"3.495,18","2.941,06","2.499,90",Disponivel,2020-09-15,20:28:24,Kabum
219,Evga,Placa de Vídeo EVGA NVIDIA GeForce RTX 2070 Su...,"4.744,59","4.705,76","3.999,90",Disponivel,2020-09-15,20:28:24,Kabum
25,Evga,Placa de Vídeo EVGA NVIDIA GeForce RTX 2070 Su...,"4.917,53","4.352,82","3.699,90",Disponivel,2020-09-15,20:28:24,Kabum
98,Galax,Placa de Vídeo Galax NVIDIA GeForce RTX 2070 S...,"5.882,24","5.529,29","4.699,90",Disponivel,2020-09-15,20:28:24,Kabum
52,Gigabyte,Placa de Vídeo Gigabyte NVIDIA GeForce GTX 165...,0,"1.705,76","1.449,90",Disponivel,2020-09-15,20:28:24,Kabum
356,Afox,"Placa de Vídeo Afox AMD Radeon RX560D, 4GB, GD...",0,"823,41","699,90",Indisponivel,2020-09-15,20:28:24,Kabum
63,Evga,Placa de Video EVGA GeForce RTX 2070 Super XC ...,"4.771,65","4.646,94","3.949,90",Disponivel,2020-09-15,20:28:24,Kabum
21,Zotac,Placa de Vídeo Zotac NVIDIA GeForce GT 710 1GB...,"381,06","352,82","299,90",Disponivel,2020-09-15,20:28:24,Kabum



### Criando a conexão com o banco e salvando os dados

In [12]:
engine = alchemy_connection('db_hardware')
df.to_sql('graphic_card', con=engine, if_exists='append', index = False)

----------------------------------------------------------------------------------------------------

In [3]:
browser2 = webdriver.Chrome(executable_path=r'chromedriver.exe', options=options)
#--| Parse or automation
browser2.get('https://www.pichau.com.br/hardware/placa-de-video?p=1')
web = soup(browser2.page_source, 'html.parser')

In [8]:
pagina = 0
html_produtos = []
flag = 0

while pagina != 4:
    items = web.findAll('div', {"class":"product-item-info"})
    if items != []:
        try:
            pagina = pagina + 1
            browser.get(f'https://www.pichau.com.br/hardware/placa-de-video?p={pagina}')
            web = soup(browser.page_source, 'html.parser')
        except Exception: break
        html_produtos.append(items)
    else:
        break

In [9]:
flat_list = []
for sublist in html_produtos:
    for item in sublist:
        flat_list.append(item)

ore-details bt-list-product" href="https://www.pichau.com.br/hardware/placa-de-video-asrock-radeon-rx-5700-xt-8gb-challenger-d-oc-256-bit-90-ga18zz-00uanz">
  <i aria-hidden="true" class="fa fa-plus"></i>
  Detalhes </a>
  <button class="action tocart bt-list-product" title="Colocar no Carrinho" type="submit">
  <i aria-hidden="true" class="fa fa-shopping-cart"></i>
  Comprar </button>
  </form>
  </div>
  </div>
  </div>
  </div>
  </div>,
  <div class="product-item-info" data-container="product-grid">
  <a class="product photo product-item-photo" href="https://www.pichau.com.br/hardware/placa-de-video-asrock-radeon-rx-5500xt-8gb-phantom-gaming-d-oc-128bit-90-ga1nzz-00uanf" onclick="window.dataLayer.push({&quot;event&quot;:&quot;productClick&quot;,&quot;eventLabel&quot;:&quot;Placa de Video ASRock Radeon RX 5500XT 8GB Phantom Gaming D OC 128BIT, 90-GA1NZZ-00UANF&quot;,&quot;ecommerce&quot;:{&quot;click&quot;:{&quot;actionField&quot;:{&quot;list&quot;:&quot;Hardware\/Placa de V\u00edde

In [ ]:
now = datetime.now()
produtos = []

for container in flat_list:
    
    # Estreutura de para salvar
    marca = container.div.img['alt']
    nome_produto = container.find('a', {"class":"sc-fzoLsD gnrNhT item-nome"}).text

    if container.find('div', {"class":"sc-fzpjYC hKlXgM"}) != None:
        preco_antigo_cartao = (container.find('div', {"class":"sc-fzpjYC hKlXgM"}).text).split(" ")[2]
    else: 
        preco_antigo_cartao = '0'
    preco_atual_cartao =  (container.find('div', {"class":"sc-fznxsB ksiZrQ"}).text).split(" ")[1]
    preco_boleto = (container.find('div', {"class":"sc-fznWqX qatGF"}).text).split(" ")[1]
    
    image = container.find('div', {"class":"sc-fznKkj fEpBwY"})
    if "indisponivel" in image.img['src']:
        disponibilidade = 'Indisponivel'
    elif "disponivel" in image.img['src']:
        disponibilidade = 'Disponivel'

    produto = {
        "marca" : marca,
        "nome_produto": nome_produto,
        "preco_antigo_cartao": preco_antigo_cartao,
        "preco_atual_cartao": preco_atual_cartao,
        "preco_boleto": preco_boleto,
        "disponibilidade": disponibilidade,
        "data" : now.strftime("%Y-%m-%d"),
        "hora":  now.strftime("%H:%M:%S"),
        "loja": "Kabum" 
    }
    produtos.append(produto)

In [15]:
# containers = html.findAll('div', {"class":"sc-fzqNqU jmuOAh"})
# https://www.kabum.com.br/cgi-local/site/listagem/listagem.cgi?string=processador&btnG=&pagina=1&ordem=5&limite=30&prime=false&marcas=[%225%22,%226%22]&tipo_produto=[]&filtro=[]

In [16]:
# images = html.findAll('div', {"class":"sc-fznKkj fEpBwY"})

# # images.img['src']
# for image in images:
#     print(image.img['src'])
#     # if "indisponivel" in image.img['src']:
#     #     print('Indisponivel')
#     # elif "disponivel" in image.img['src']:
#     #     print('Disponivel')

In [17]:
# preco_antigo_cartao = float((container.find('div', {"class":"sc-fzpjYC hKlXgM"}).text).split(" ")[2].replace('.', '').replace(',', '.'))
# preco_atual_cartao =  float((container.find('div', {"class":"sc-fznxsB ksiZrQ"}).text).split(" ")[1].replace('.', '').replace(',', '.'))
# preco_boleto = float((container.find('div', {"class":"sc-fznWqX qatGF"}).text).split(" ")[1].replace('.', '').replace(',', '.'))


In [18]:
# print(preco_antigo_cartao)
# # print(re.search('^(([1-9]\d{0,2}(\.\d{3})*)|(([1-9]\.\d*)?\d))(\,\d\d)?', preco_atual_cartao).group())
# print(preco_atual_cartao)
# print(preco_boleto)